# Lesson I2: Error Handling & Traits

**Duration**: 210-240 minutes  
**Stage**: Intermediate (Building Skills)  
**Prerequisites**: Lesson I1 (Structs & Methods)

---

## 📋 What You'll Learn

This comprehensive lesson covers two fundamental Rust concepts: error handling and traits. You'll learn to handle errors gracefully using Result types and combinators, then master traits - Rust's approach to polymorphism and shared behavior.

**Why this matters**: Rust's error handling eliminates exceptions and forces explicit error handling, preventing crashes. Traits enable code reuse and abstraction without runtime overhead, making them essential for library design and generic programming.

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Use Result<T,E> for error handling
2. Implement custom error types
3. Use the ? operator and Result combinators for error propagation
4. Define traits to specify shared behavior across types
5. Implement traits for custom and existing types
6. Use trait bounds to constrain generic parameters
7. Work with trait objects and understand static vs dynamic dispatch
8. Create generic functions and types
9. Understand monomorphization and object safety


In [ ]:
use std::fmt;

// Simple enum-based error type
#[derive(Debug, Clone)]
enum MathError {
    DivisionByZero,
    NegativeSquareRoot,
    InvalidInput(String),
    Overflow,
}

// Implement Display for user-friendly error messages
impl fmt::Display for MathError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            MathError::DivisionByZero => write!(f, "Cannot divide by zero"),
            MathError::NegativeSquareRoot => write!(f, "Cannot take square root of negative number"),
            MathError::InvalidInput(msg) => write!(f, "Invalid input: {}", msg),
            MathError::Overflow => write!(f, "Mathematical overflow occurred"),
        }
    }
}

// Implement std::error::Error trait
impl std::error::Error for MathError {}

// Math operations with error handling
fn safe_divide(a: f64, b: f64) -> Result<f64, MathError> {
    if b == 0.0 {
        Err(MathError::DivisionByZero)
    } else if a.is_infinite() || b.is_infinite() {
        Err(MathError::Overflow)
    } else {
        Ok(a / b)
    }
}

fn safe_sqrt(x: f64) -> Result<f64, MathError> {
    if x < 0.0 {
        Err(MathError::NegativeSquareRoot)
    } else if x.is_infinite() {
        Err(MathError::Overflow)
    } else {
        Ok(x.sqrt())
    }
}

fn parse_and_sqrt(input: &str) -> Result<f64, MathError> {
    let number: f64 = input.parse()
        .map_err(|_| MathError::InvalidInput(format!("'{}' is not a valid number", input)))?;
    
    safe_sqrt(number)
}

fn custom_error_demo() {
    println!("=== Custom Error Types Demo ===");
    
    let test_cases = vec![
        ("25", "Valid input"),
        ("-4", "Negative number"),
        ("abc", "Invalid format"),
        ("16", "Valid input"),
    ];
    
    for (input, description) in test_cases {
        print!("Testing '{}' ({}): ", input, description);
        
        match parse_and_sqrt(input) {
            Ok(result) => println!("✅ √{} = {:.2}", input, result),
            Err(e) => println!("❌ Error: {}", e),
        }
    }
    
    // Division examples
    println!("\n=== Division Examples ===");
    let divisions = vec![(10.0, 2.0), (5.0, 0.0), (f64::INFINITY, 2.0)];
    
    for (a, b) in divisions {
        match safe_divide(a, b) {
            Ok(result) => println!("✅ {} ÷ {} = {:.2}", a, b, result),
            Err(e) => println!("❌ {} ÷ {} failed: {}", a, b, e),
        }
    }
}

custom_error_demo();

In [ ]:
// Complex calculation that can fail at multiple steps
fn complex_calculation(a: f64, b: f64, c: f64) -> Result<f64, MathError> {
    // Step 1: Calculate discriminant
    let discriminant = b * b - 4.0 * a * c;
    
    // Step 2: Take square root (can fail)
    let sqrt_discriminant = safe_sqrt(discriminant)?;
    
    // Step 3: Calculate numerator
    let numerator = -b + sqrt_discriminant;
    
    // Step 4: Calculate denominator
    let denominator = 2.0 * a;
    
    // Step 5: Divide (can fail)
    let result = safe_divide(numerator, denominator)?;
    
    Ok(result)
}

// Alternative without ? operator (more verbose)
fn complex_calculation_verbose(a: f64, b: f64, c: f64) -> Result<f64, MathError> {
    let discriminant = b * b - 4.0 * a * c;
    
    let sqrt_discriminant = match safe_sqrt(discriminant) {
        Ok(val) => val,
        Err(e) => return Err(e),
    };
    
    let numerator = -b + sqrt_discriminant;
    let denominator = 2.0 * a;
    
    let result = match safe_divide(numerator, denominator) {
        Ok(val) => val,
        Err(e) => return Err(e),
    };
    
    Ok(result)
}

// Chain multiple operations
fn process_numbers(inputs: Vec<&str>) -> Result<Vec<f64>, MathError> {
    let mut results = Vec::new();
    
    for input in inputs {
        let number = input.parse::<f64>()
            .map_err(|_| MathError::InvalidInput(format!("Invalid number: {}", input)))?;
        
        let sqrt_result = safe_sqrt(number)?;
        results.push(sqrt_result);
    }
    
    Ok(results)
}

fn error_propagation_demo() {
    println!("=== Error Propagation Demo ===");
    
    // Quadratic formula examples
    let equations = vec![
        (1.0, -5.0, 6.0, "x² - 5x + 6 = 0"),
        (1.0, 2.0, 5.0, "x² + 2x + 5 = 0 (negative discriminant)"),
        (0.0, 1.0, 1.0, "0x² + x + 1 = 0 (division by zero)"),
    ];
    
    for (a, b, c, description) in equations {
        println!("\nSolving: {}", description);
        
        match complex_calculation(a, b, c) {
            Ok(root) => println!("✅ One root: {:.2}", root),
            Err(e) => println!("❌ Error: {}", e),
        }
    }
    
    // Processing multiple inputs
    println!("\n=== Processing Multiple Inputs ===");
    
    let test_inputs = vec![
        vec!["4", "9", "16", "25"],
        vec!["1", "4", "-9", "16"],  // Contains negative number
        vec!["1", "abc", "16"],       // Contains invalid input
    ];
    
    for (i, inputs) in test_inputs.iter().enumerate() {
        println!("\nTest set {}: {:?}", i + 1, inputs);
        
        match process_numbers(inputs.clone()) {
            Ok(results) => {
                println!("✅ Square roots: {:?}", 
                        results.iter().map(|x| format!("{:.2}", x)).collect::<Vec<_>>());
            },
            Err(e) => println!("❌ Processing failed: {}", e),
        }
    }
}

error_propagation_demo();

---

## 🔗 Result Combinators

Rust provides powerful combinator methods for working with `Result` and `Option` types without explicit pattern matching.

### Common Result Combinators

| Combinator | Signature | Description |
|------------|-----------|-------------|
| `map` | `Result<T, E> -> Result<U, E>` | Transform success value |
| `map_err` | `Result<T, E> -> Result<T, F>` | Transform error value |
| `and_then` | `Result<T, E> -> Result<U, E>` | Chain fallible operations |
| `or_else` | `Result<T, E> -> Result<T, F>` | Provide fallback on error |
| `unwrap_or` | `Result<T, E> -> T` | Provide default value |
| `unwrap_or_else` | `Result<T, E> -> T` | Compute default value |
| `ok` | `Result<T, E> -> Option<T>` | Convert to Option |

**📚 Rust Book Reference:** [Chapter 9.2 - Recoverable Errors with Result](https://doc.rust-lang.org/book/ch09-02-recoverable-errors-with-result.html)

In [ ]:
// Demonstrating Result combinators

fn parse_number(s: &str) -> Result<i32, String> {
    s.parse::<i32>()
        .map_err(|e| format!("Failed to parse '{}': {}", s, e))
}

fn double_if_positive(n: i32) -> Result<i32, String> {
    if n > 0 {
        Ok(n * 2)
    } else {
        Err(format!("Number {} is not positive", n))
    }
}

fn result_combinators_demo() {
    println!("=== Result Combinators Demo ===");
    
    // map: Transform success value
    let result1 = parse_number("42")
        .map(|n| n + 10);  // Add 10 if parsing succeeds
    println!("map: {:?}", result1);  // Ok(52)
    
    // and_then: Chain operations that can fail
    let result2 = parse_number("5")
        .and_then(double_if_positive);  // Parse, then double if positive
    println!("and_then (positive): {:?}", result2);  // Ok(10)
    
    let result3 = parse_number("-5")
        .and_then(double_if_positive);
    println!("and_then (negative): {:?}", result3);  // Err(...)
    
    // or_else: Provide fallback on error
    let result4 = parse_number("invalid")
        .or_else(|_| Ok(0));  // Use 0 as fallback
    println!("or_else: {:?}", result4);  // Ok(0)
    
    // unwrap_or: Provide default value
    let value1 = parse_number("100").unwrap_or(0);
    let value2 = parse_number("bad").unwrap_or(0);
    println!("unwrap_or: {} and {}", value1, value2);  // 100 and 0
    
    // unwrap_or_else: Compute default value (lazy evaluation)
    let value3 = parse_number("bad")
        .unwrap_or_else(|e| {
            println!("Error occurred: {}", e);
            -1  // Return -1 on error
        });
    println!("unwrap_or_else: {}", value3);
    
    // Chaining multiple combinators
    let result5 = parse_number("8")
        .map(|n| n * 2)           // Double it
        .and_then(double_if_positive)  // Double again if positive
        .map(|n| n + 1);          // Add 1
    println!("Chained: {:?}", result5);  // Ok(33)
}

result_combinators_demo();

In [ ]:
use std::num::ParseIntError;
use std::io;

// Application-specific error that can wrap multiple error types
#[derive(Debug)]
enum AppError {
    Math(MathError),
    Parse(ParseIntError),
    Io(io::Error),
    Custom(String),
}

impl fmt::Display for AppError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            AppError::Math(e) => write!(f, "Math error: {}", e),
            AppError::Parse(e) => write!(f, "Parse error: {}", e),
            AppError::Io(e) => write!(f, "IO error: {}", e),
            AppError::Custom(msg) => write!(f, "Application error: {}", msg),
        }
    }
}

impl std::error::Error for AppError {
    fn source(&self) -> Option<&(dyn std::error::Error + 'static)> {
        match self {
            AppError::Math(e) => Some(e),
            AppError::Parse(e) => Some(e),
            AppError::Io(e) => Some(e),
            AppError::Custom(_) => None,
        }
    }
}

// Automatic conversion from specific error types
impl From<MathError> for AppError {
    fn from(error: MathError) -> Self {
        AppError::Math(error)
    }
}

impl From<ParseIntError> for AppError {
    fn from(error: ParseIntError) -> Self {
        AppError::Parse(error)
    }
}

impl From<io::Error> for AppError {
    fn from(error: io::Error) -> Self {
        AppError::Io(error)
    }
}

// Function that can produce multiple types of errors
fn process_input_string(input: &str) -> Result<f64, AppError> {
    // This can produce a ParseIntError
    let number: i32 = input.parse()?;  // Automatically converted to AppError
    
    // Validate range
    if number < 0 {
        return Err(AppError::Custom("Number must be non-negative".to_string()));
    }
    
    if number > 1000 {
        return Err(AppError::Custom("Number too large (max 1000)".to_string()));
    }
    
    // This can produce a MathError
    let result = safe_sqrt(number as f64)?;  // Automatically converted to AppError
    
    Ok(result)
}

// Batch processing with detailed error reporting
fn batch_process(inputs: Vec<&str>) -> Result<Vec<f64>, Vec<(usize, AppError)>> {
    let mut results = Vec::new();
    let mut errors = Vec::new();
    
    for (index, input) in inputs.iter().enumerate() {
        match process_input_string(input) {
            Ok(result) => results.push(result),
            Err(e) => errors.push((index, e)),
        }
    }
    
    if errors.is_empty() {
        Ok(results)
    } else {
        Err(errors)
    }
}

fn multiple_error_types_demo() {
    println!("=== Multiple Error Types Demo ===");
    
    let test_inputs = vec![
        "25",      // Valid
        "abc",     // Parse error
        "-5",      // Custom error (negative)
        "1500",    // Custom error (too large)
        "100",     // Valid
        "0",       // Valid (edge case)
    ];
    
    println!("Processing individual inputs:");
    for input in &test_inputs {
        match process_input_string(input) {
            Ok(result) => println!("✅ '{}' -> √{} = {:.2}", input, input, result),
            Err(e) => println!("❌ '{}' -> {}", input, e),
        }
    }
    
    println!("\nBatch processing:");
    match batch_process(test_inputs) {
        Ok(results) => {
            println!("✅ All inputs processed successfully:");
            for (i, result) in results.iter().enumerate() {
                println!("   Result {}: {:.2}", i + 1, result);
            }
        },
        Err(errors) => {
            println!("❌ Batch processing failed with {} errors:", errors.len());
            for (index, error) in errors {
                println!("   Input {}: {}", index + 1, error);
            }
        },
    }
}

multiple_error_types_demo();

In [ ]:
// Different strategies for handling errors

// Strategy 1: Fail fast - propagate errors immediately
fn fail_fast_strategy(inputs: Vec<&str>) -> Result<f64, AppError> {
    let mut sum = 0.0;
    
    for input in inputs {
        let value = process_input_string(input)?;  // Fail on first error
        sum += value;
    }
    
    Ok(sum)
}

// Strategy 2: Collect all errors
fn collect_errors_strategy(inputs: Vec<&str>) -> Result<f64, Vec<AppError>> {
    let mut sum = 0.0;
    let mut errors = Vec::new();
    
    for input in inputs {
        match process_input_string(input) {
            Ok(value) => sum += value,
            Err(e) => errors.push(e),
        }
    }
    
    if errors.is_empty() {
        Ok(sum)
    } else {
        Err(errors)
    }
}

// Strategy 3: Skip errors with logging
fn skip_errors_strategy(inputs: Vec<&str>) -> (f64, Vec<String>) {
    let mut sum = 0.0;
    let mut warnings = Vec::new();
    
    for input in inputs {
        match process_input_string(input) {
            Ok(value) => sum += value,
            Err(e) => {
                warnings.push(format!("Skipped '{}': {}", input, e));
            },
        }
    }
    
    (sum, warnings)
}

// Strategy 4: Retry with fallback
fn retry_with_fallback(input: &str, fallback: f64) -> f64 {
    // Try primary processing
    if let Ok(result) = process_input_string(input) {
        return result;
    }
    
    // Try alternative parsing (as float directly)
    if let Ok(number) = input.parse::<f64>() {
        if number >= 0.0 {
            if let Ok(result) = safe_sqrt(number) {
                return result;
            }
        }
    }
    
    // Use fallback
    fallback
}

fn error_strategies_demo() {
    println!("=== Error Handling Strategies Demo ===");
    
    let test_data = vec!["4", "abc", "16", "-5", "25"];
    
    println!("Test data: {:?}\n", test_data);
    
    // Strategy 1: Fail fast
    println!("Strategy 1 - Fail Fast:");
    match fail_fast_strategy(test_data.clone()) {
        Ok(sum) => println!("✅ Sum: {:.2}", sum),
        Err(e) => println!("❌ Failed: {}", e),
    }
    
    // Strategy 2: Collect all errors
    println!("\nStrategy 2 - Collect All Errors:");
    match collect_errors_strategy(test_data.clone()) {
        Ok(sum) => println!("✅ Sum: {:.2}", sum),
        Err(errors) => {
            println!("❌ {} errors occurred:", errors.len());
            for (i, error) in errors.iter().enumerate() {
                println!("   {}: {}", i + 1, error);
            }
        },
    }
    
    // Strategy 3: Skip errors
    println!("\nStrategy 3 - Skip Errors:");
    let (sum, warnings) = skip_errors_strategy(test_data.clone());
    println!("✅ Sum of valid inputs: {:.2}", sum);
    if !warnings.is_empty() {
        println!("⚠️  Warnings:");
        for warning in warnings {
            println!("   {}", warning);
        }
    }
    
    // Strategy 4: Retry with fallback
    println!("\nStrategy 4 - Retry with Fallback:");
    let fallback_value = 1.0;
    let mut sum = 0.0;
    
    for input in &test_data {
        let result = retry_with_fallback(input, fallback_value);
        println!("  '{}' -> {:.2}", input, result);
        sum += result;
    }
    
    println!("✅ Total sum with fallbacks: {:.2}", sum);
}

error_strategies_demo();

In [ ]:
// TODO: Complete the file processing system

#[derive(Debug)]
enum FileProcessingError {
    InvalidFormat(String),
    EmptyFile,
    ProcessingError(String),
    ValidationError(String),
}

impl fmt::Display for FileProcessingError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            FileProcessingError::InvalidFormat(msg) => write!(f, "Invalid format: {}", msg),
            FileProcessingError::EmptyFile => write!(f, "File is empty"),
            FileProcessingError::ProcessingError(msg) => write!(f, "Processing error: {}", msg),
            FileProcessingError::ValidationError(msg) => write!(f, "Validation error: {}", msg),
        }
    }
}

impl std::error::Error for FileProcessingError {}

#[derive(Debug, Clone)]
struct DataRecord {
    id: u32,
    name: String,
    value: f64,
}

impl DataRecord {
    // TODO: Parse a CSV line into a DataRecord
    fn from_csv_line(line: &str, line_number: usize) -> Result<Self, FileProcessingError> {
        let parts: Vec<&str> = line.split(',').collect();
        
        if parts.len() != 3 {
            return Err(FileProcessingError::InvalidFormat(
                format!("Line {}: Expected 3 fields, found {}", line_number, parts.len())
            ));
        }
        
        let id = parts[0].trim().parse::<u32>()
            .map_err(|_| FileProcessingError::InvalidFormat(
                format!("Line {}: Invalid ID '{}'", line_number, parts[0])
            ))?;
        
        let name = parts[1].trim().to_string();
        if name.is_empty() {
            return Err(FileProcessingError::ValidationError(
                format!("Line {}: Name cannot be empty", line_number)
            ));
        }
        
        let value = parts[2].trim().parse::<f64>()
            .map_err(|_| FileProcessingError::InvalidFormat(
                format!("Line {}: Invalid value '{}'", line_number, parts[2])
            ))?;
        
        if value < 0.0 {
            return Err(FileProcessingError::ValidationError(
                format!("Line {}: Value must be non-negative, got {}", line_number, value)
            ));
        }
        
        Ok(DataRecord { id, name, value })
    }
    
    // TODO: Validate the record
    fn validate(&self) -> Result<(), FileProcessingError> {
        if self.name.len() < 2 {
            return Err(FileProcessingError::ValidationError(
                format!("Name '{}' is too short (minimum 2 characters)", self.name)
            ));
        }
        
        if self.value > 1000.0 {
            return Err(FileProcessingError::ValidationError(
                format!("Value {} exceeds maximum allowed (1000.0)", self.value)
            ));
        }
        
        Ok(())
    }
}

// TODO: Process CSV data with comprehensive error handling
fn process_csv_data(csv_content: &str) -> Result<Vec<DataRecord>, Vec<FileProcessingError>> {
    let lines: Vec<&str> = csv_content.lines().collect();
    
    if lines.is_empty() {
        return Err(vec![FileProcessingError::EmptyFile]);
    }
    
    let mut records = Vec::new();
    let mut errors = Vec::new();
    
    // Skip header if present
    let data_lines = if lines[0].to_lowercase().contains("id") {
        &lines[1..]
    } else {
        &lines[..]
    };
    
    for (index, line) in data_lines.iter().enumerate() {
        let line_number = index + 2; // Account for header and 1-based indexing
        
        if line.trim().is_empty() {
            continue; // Skip empty lines
        }
        
        match DataRecord::from_csv_line(line, line_number) {
            Ok(record) => {
                match record.validate() {
                    Ok(()) => records.push(record),
                    Err(e) => errors.push(e),
                }
            },
            Err(e) => errors.push(e),
        }
    }
    
    if errors.is_empty() {
        Ok(records)
    } else {
        Err(errors)
    }
}

fn file_processing_demo() {
    println!("=== File Processing Demo ===");
    
    // Sample CSV data with various issues
    let csv_data = r#"ID,Name,Value
1,Alice,25.5
2,Bob,30.0
abc,Charlie,15.0
4,,20.0
5,David,-5.0
6,Eve,1500.0
7,Frank,45.5
8,Grace
9,Henry,35.0
10,A,10.0"#;
    
    println!("Processing CSV data:");
    println!("{}", csv_data);
    println!("\nResults:");
    
    match process_csv_data(csv_data) {
        Ok(records) => {
            println!("✅ Successfully processed {} records:", records.len());
            for record in records {
                println!("   ID: {}, Name: {}, Value: {:.2}", 
                        record.id, record.name, record.value);
            }
        },
        Err(errors) => {
            println!("❌ Processing failed with {} errors:", errors.len());
            for (i, error) in errors.iter().enumerate() {
                println!("   {}: {}", i + 1, error);
            }
        },
    }
    
    // Test with empty data
    println!("\n=== Testing Empty Data ===");
    match process_csv_data("") {
        Ok(_) => println!("✅ Empty data processed successfully"),
        Err(errors) => {
            for error in errors {
                println!("❌ {}", error);
            }
        },
    }
}

file_processing_demo();

In [ ]:
// Generic function fundamentals
fn generic_functions_demo() {
    println!("=== Generic Functions ===");
    
    // Simple generic function
    fn largest<T: PartialOrd + Copy>(list: &[T]) -> T {
        let mut largest = list[0];
        
        for &item in list {
            if item > largest {
                largest = item;
            }
        }
        
        largest
    }
    
    // Works with different types
    let numbers = vec![34, 50, 25, 100, 65];
    let result = largest(&numbers);
    println!("Largest number: {}", result);
    
    let chars = vec!['y', 'm', 'a', 'q'];
    let result = largest(&chars);
    println!("Largest char: {}", result);
    
    // Generic function with multiple type parameters
    fn make_pair<T, U>(first: T, second: U) -> (T, U) {
        (first, second)
    }
    
    let pair1 = make_pair(42, "hello");
    let pair2 = make_pair(3.14, true);
    
    println!("\nPairs created:");
    println!("Pair 1: {:?}", pair1);
    println!("Pair 2: {:?}", pair2);
    
    // Generic function with constraints
    fn print_and_return<T: std::fmt::Display + Clone>(value: T) -> T {
        println!("Value: {}", value);
        value.clone()
    }
    
    let number = print_and_return(42);
    let text = print_and_return("Rust".to_string());
    
    println!("\nReturned values: {} and {}", number, text);
    
    // Generic function with where clause
    fn compare_and_display<T, U>(t: &T, u: &U) -> bool
    where
        T: std::fmt::Display + PartialEq<U>,
        U: std::fmt::Display,
    {
        println!("Comparing {} and {}", t, u);
        t == u
    }
    
    let result = compare_and_display(&5, &5);
    println!("Are they equal? {}", result);
}

generic_functions_demo();

In [ ]:
// Generic structs and implementations

#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn new(x: T, y: T) -> Self {
        Point { x, y }
    }
    
    fn x(&self) -> &T {
        &self.x
    }
    
    fn y(&self) -> &T {
        &self.y
    }
}

// Implementation for specific types
impl Point<f32> {
    fn distance_from_origin(&self) -> f32 {
        (self.x.powi(2) + self.y.powi(2)).sqrt()
    }
}

// Generic struct with multiple type parameters
#[derive(Debug)]
struct Pair<T, U> {
    first: T,
    second: U,
}

impl<T, U> Pair<T, U> {
    fn new(first: T, second: U) -> Self {
        Pair { first, second }
    }
    
    fn first(&self) -> &T {
        &self.first
    }
    
    fn second(&self) -> &U {
        &self.second
    }
    
    // Method that transforms the pair
    fn map<V, W, F, G>(self, f: F, g: G) -> Pair<V, W>
    where
        F: FnOnce(T) -> V,
        G: FnOnce(U) -> W,
    {
        Pair {
            first: f(self.first),
            second: g(self.second),
        }
    }
}

// Generic struct with constraints
#[derive(Debug)]
struct Container<T: Clone + std::fmt::Display> {
    items: Vec<T>,
}

impl<T: Clone + std::fmt::Display> Container<T> {
    fn new() -> Self {
        Container { items: Vec::new() }
    }
    
    fn add(&mut self, item: T) {
        self.items.push(item);
    }
    
    fn display_all(&self) {
        for (i, item) in self.items.iter().enumerate() {
            println!("  Item {}: {}", i, item);
        }
    }
    
    fn duplicate_last(&mut self) -> Option<()> {
        if let Some(last) = self.items.last() {
            self.items.push(last.clone());
            Some(())
        } else {
            None
        }
    }
}

fn generic_structs_demo() {
    println!("\n=== Generic Structs ===");
    
    // Point with different types
    let integer_point = Point::new(5, 10);
    let float_point = Point::new(1.0, 4.0);
    
    println!("Integer point: {:?}", integer_point);
    println!("Float point: {:?}", float_point);
    println!("Distance from origin: {:.2}", float_point.distance_from_origin());
    
    // Pair with different types
    let mixed_pair = Pair::new("Hello", 42);
    println!("\nMixed pair: {:?}", mixed_pair);
    
    // Transform the pair
    let transformed = mixed_pair.map(
        |s| s.len(),
        |n| n as f64 * 2.5
    );
    println!("Transformed pair: {:?}", transformed);
    
    // Container with constraints
    let mut string_container = Container::new();
    string_container.add("Rust".to_string());
    string_container.add("is".to_string());
    string_container.add("awesome".to_string());
    
    println!("\nString container:");
    string_container.display_all();
    
    string_container.duplicate_last();
    println!("\nAfter duplicating last:");
    string_container.display_all();
    
    let mut number_container = Container::new();
    number_container.add(1);
    number_container.add(2);
    number_container.add(3);
    
    println!("\nNumber container:");
    number_container.display_all();
}

generic_structs_demo();

In [ ]:
// Generic enums and advanced patterns

#[derive(Debug)]
enum Result<T, E> {
    Ok(T),
    Err(E),
}

impl<T, E> Result<T, E> {
    fn is_ok(&self) -> bool {
        matches!(self, Result::Ok(_))
    }
    
    fn is_err(&self) -> bool {
        matches!(self, Result::Err(_))
    }
    
    fn map<U, F>(self, f: F) -> Result<U, E>
    where
        F: FnOnce(T) -> U,
    {
        match self {
            Result::Ok(value) => Result::Ok(f(value)),
            Result::Err(error) => Result::Err(error),
        }
    }
    
    fn and_then<U, F>(self, f: F) -> Result<U, E>
    where
        F: FnOnce(T) -> Result<U, E>,
    {
        match self {
            Result::Ok(value) => f(value),
            Result::Err(error) => Result::Err(error),
        }
    }
}

// Generic enum for binary tree
#[derive(Debug)]
enum BinaryTree<T> {
    Empty,
    Node {
        value: T,
        left: Box<BinaryTree<T>>,
        right: Box<BinaryTree<T>>,
    },
}

impl<T> BinaryTree<T> {
    fn new() -> Self {
        BinaryTree::Empty
    }
    
    fn leaf(value: T) -> Self {
        BinaryTree::Node {
            value,
            left: Box::new(BinaryTree::Empty),
            right: Box::new(BinaryTree::Empty),
        }
    }
    
    fn node(value: T, left: BinaryTree<T>, right: BinaryTree<T>) -> Self {
        BinaryTree::Node {
            value,
            left: Box::new(left),
            right: Box::new(right),
        }
    }
    
    fn size(&self) -> usize {
        match self {
            BinaryTree::Empty => 0,
            BinaryTree::Node { left, right, .. } => 1 + left.size() + right.size(),
        }
    }
}

impl<T: std::fmt::Display> BinaryTree<T> {
    fn print_inorder(&self) {
        match self {
            BinaryTree::Empty => {},
            BinaryTree::Node { value, left, right } => {
                left.print_inorder();
                print!("{} ", value);
                right.print_inorder();
            },
        }
    }
}

// Generic enum for different data types
#[derive(Debug)]
enum Either<L, R> {
    Left(L),
    Right(R),
}

impl<L, R> Either<L, R> {
    fn is_left(&self) -> bool {
        matches!(self, Either::Left(_))
    }
    
    fn is_right(&self) -> bool {
        matches!(self, Either::Right(_))
    }
    
    fn map_left<T, F>(self, f: F) -> Either<T, R>
    where
        F: FnOnce(L) -> T,
    {
        match self {
            Either::Left(value) => Either::Left(f(value)),
            Either::Right(value) => Either::Right(value),
        }
    }
    
    fn map_right<T, F>(self, f: F) -> Either<L, T>
    where
        F: FnOnce(R) -> T,
    {
        match self {
            Either::Left(value) => Either::Left(value),
            Either::Right(value) => Either::Right(f(value)),
        }
    }
}

fn generic_enums_demo() {
    println!("\n=== Generic Enums ===");
    
    // Custom Result type
    fn divide(a: f64, b: f64) -> Result<f64, String> {
        if b == 0.0 {
            Result::Err("Division by zero".to_string())
        } else {
            Result::Ok(a / b)
        }
    }
    
    let result1 = divide(10.0, 2.0);
    let result2 = divide(10.0, 0.0);
    
    println!("Division results:");
    println!("  10.0 / 2.0 = {:?}", result1);
    println!("  10.0 / 0.0 = {:?}", result2);
    
    // Chain operations with map and and_then
    let chained = divide(20.0, 4.0)
        .map(|x| x * 2.0)
        .and_then(|x| divide(x, 2.0));
    
    println!("  Chained result: {:?}", chained);
    
    // Binary tree
    let tree = BinaryTree::node(
        4,
        BinaryTree::node(
            2,
            BinaryTree::leaf(1),
            BinaryTree::leaf(3)
        ),
        BinaryTree::node(
            6,
            BinaryTree::leaf(5),
            BinaryTree::leaf(7)
        )
    );
    
    println!("\nBinary tree:");
    println!("  Size: {}", tree.size());
    print!("  Inorder traversal: ");
    tree.print_inorder();
    println!();
    
    // Either type for handling different data
    let values: Vec<Either<i32, String>> = vec![
        Either::Left(42),
        Either::Right("Hello".to_string()),
        Either::Left(100),
        Either::Right("World".to_string()),
    ];
    
    println!("\nEither values:");
    for (i, value) in values.iter().enumerate() {
        match value {
            Either::Left(num) => println!("  {}: Number {}", i, num),
            Either::Right(text) => println!("  {}: Text '{}'", i, text),
        }
    }
    
    // Transform Either values
    let transformed: Vec<_> = values.into_iter()
        .map(|either| either.map_left(|n| n * 2).map_right(|s| s.to_uppercase()))
        .collect();
    
    println!("\nTransformed values: {:?}", transformed);
}

generic_enums_demo();

In [ ]:
// TODO: Complete the generic stack implementation

#[derive(Debug)]
struct Stack<T> {
    items: Vec<T>,
}

impl<T> Stack<T> {
    fn new() -> Self {
        // TODO: Create a new empty stack
        Stack { items: Vec::new() }
    }
    
    fn push(&mut self, item: T) {
        // TODO: Push an item onto the stack
        self.items.push(item);
    }
    
    fn pop(&mut self) -> Option<T> {
        // TODO: Pop an item from the stack
        self.items.pop()
    }
    
    fn peek(&self) -> Option<&T> {
        // TODO: Look at the top item without removing it
        self.items.last()
    }
    
    fn is_empty(&self) -> bool {
        // TODO: Check if the stack is empty
        self.items.is_empty()
    }
    
    fn size(&self) -> usize {
        // TODO: Return the number of items in the stack
        self.items.len()
    }
    
    fn clear(&mut self) {
        // TODO: Remove all items from the stack
        self.items.clear();
    }
}

// Additional methods with constraints
impl<T: Clone> Stack<T> {
    fn duplicate_top(&mut self) -> Option<()> {
        // TODO: Duplicate the top item (requires Clone)
        if let Some(top) = self.items.last() {
            let cloned = top.clone();
            self.items.push(cloned);
            Some(())
        } else {
            None
        }
    }
    
    fn to_vec(&self) -> Vec<T> {
        // TODO: Convert stack to vector (top to bottom order)
        let mut result = self.items.clone();
        result.reverse();
        result
    }
}

impl<T: std::fmt::Display> Stack<T> {
    fn display(&self) {
        // TODO: Display the stack contents (requires Display)
        println!("Stack (top to bottom):");
        for (i, item) in self.items.iter().rev().enumerate() {
            println!("  {}: {}", i, item);
        }
        if self.items.is_empty() {
            println!("  (empty)");
        }
    }
}

impl<T: PartialEq> Stack<T> {
    fn contains(&self, item: &T) -> bool {
        // TODO: Check if stack contains an item (requires PartialEq)
        self.items.contains(item)
    }
    
    fn remove_all(&mut self, item: &T) -> usize {
        // TODO: Remove all occurrences of an item, return count removed
        let original_len = self.items.len();
        self.items.retain(|x| x != item);
        original_len - self.items.len()
    }
}

fn stack_demo() {
    println!("\n=== Generic Stack Demo ===");
    
    // Integer stack
    let mut int_stack = Stack::new();
    
    // Push some numbers
    for i in 1..=5 {
        int_stack.push(i);
    }
    
    println!("Integer stack after pushing 1-5:");
    int_stack.display();
    
    // Pop and peek operations
    if let Some(top) = int_stack.peek() {
        println!("\nTop item (peek): {}", top);
    }
    
    if let Some(popped) = int_stack.pop() {
        println!("Popped item: {}", popped);
    }
    
    int_stack.display();
    
    // Duplicate top
    int_stack.duplicate_top();
    println!("\nAfter duplicating top:");
    int_stack.display();
    
    // String stack
    let mut string_stack = Stack::new();
    string_stack.push("first".to_string());
    string_stack.push("second".to_string());
    string_stack.push("third".to_string());
    string_stack.push("second".to_string()); // Duplicate
    
    println!("\nString stack:");
    string_stack.display();
    
    // Check contains and remove
    let search_item = "second".to_string();
    println!("\nContains '{}': {}", search_item, string_stack.contains(&search_item));
    
    let removed_count = string_stack.remove_all(&search_item);
    println!("Removed {} occurrences of '{}'", removed_count, search_item);
    
    string_stack.display();
    
    // Convert to vector
    let as_vec = string_stack.to_vec();
    println!("\nAs vector (top to bottom): {:?}", as_vec);
    
    // Stack statistics
    println!("\nStack statistics:");
    println!("  Size: {}", string_stack.size());
    println!("  Is empty: {}", string_stack.is_empty());
    
    string_stack.clear();
    println!("\nAfter clearing:");
    string_stack.display();
    println!("  Is empty: {}", string_stack.is_empty());
}

stack_demo();

---

# Part 2: Content from Second Lesson

---


In [ ]:
// Basic trait definition and implementation

trait Drawable {
    fn draw(&self);
    fn area(&self) -> f64;
    
    // Default implementation
    fn describe(&self) {
        println!("This shape has an area of {:.2}", self.area());
    }
}

struct Circle {
    radius: f64,
}

impl Circle {
    fn new(radius: f64) -> Self {
        Circle { radius }
    }
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing a circle with radius {}", self.radius);
    }
    
    fn area(&self) -> f64 {
        std::f64::consts::PI * self.radius * self.radius
    }
}

struct Rectangle {
    width: f64,
    height: f64,
}

impl Rectangle {
    fn new(width: f64, height: f64) -> Self {
        Rectangle { width, height }
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
        println!("Drawing a rectangle {}x{}", self.width, self.height);
    }
    
    fn area(&self) -> f64 {
        self.width * self.height
    }
    
    // Override default implementation
    fn describe(&self) {
        println!("Rectangle: {}x{} with area {:.2}", 
                self.width, self.height, self.area());
    }
}

// Function that works with any Drawable
fn draw_shape(shape: &dyn Drawable) {
    shape.draw();
    shape.describe();
}

// Generic function with trait bound
fn print_area<T: Drawable>(shape: &T) {
    println!("Area: {:.2}", shape.area());
}

fn basic_traits_demo() {
    println!("=== Basic Traits Demo ===");
    
    let circle = Circle::new(5.0);
    let rectangle = Rectangle::new(4.0, 6.0);
    
    // Direct method calls
    circle.draw();
    circle.describe();
    
    rectangle.draw();
    rectangle.describe();
    
    println!();
    
    // Using trait objects
    let shapes: Vec<&dyn Drawable> = vec![&circle, &rectangle];
    
    for (i, shape) in shapes.iter().enumerate() {
        println!("Shape {}:", i + 1);
        draw_shape(*shape);
        println!();
    }
    
    // Using generic function
    print_area(&circle);
    print_area(&rectangle);
}

basic_traits_demo();

In [ ]:
// Advanced trait features: associated types, constants, and complex bounds

trait Iterator {
    type Item; // Associated type
    
    fn next(&mut self) -> Option<Self::Item>;
    
    // Default implementation using associated type
    fn collect<C: FromIterator<Self::Item>>(self) -> C
    where
        Self: Sized,
    {
        FromIterator::from_iter(self)
    }
}

trait FromIterator<T> {
    fn from_iter<I: Iterator<Item = T>>(iter: I) -> Self;
}

// Custom iterator implementation
struct Counter {
    current: usize,
    max: usize,
}

impl Counter {
    fn new(max: usize) -> Self {
        Counter { current: 0, max }
    }
}

impl Iterator for Counter {
    type Item = usize;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.current < self.max {
            let current = self.current;
            self.current += 1;
            Some(current)
        } else {
            None
        }
    }
}

// Trait with associated constants
trait Numeric {
    const ZERO: Self;
    const ONE: Self;
    
    fn add(self, other: Self) -> Self;
    fn multiply(self, other: Self) -> Self;
    
    fn square(self) -> Self
    where
        Self: Sized + Copy,
    {
        self.multiply(self)
    }
}

impl Numeric for i32 {
    const ZERO: Self = 0;
    const ONE: Self = 1;
    
    fn add(self, other: Self) -> Self {
        self + other
    }
    
    fn multiply(self, other: Self) -> Self {
        self * other
    }
}

impl Numeric for f64 {
    const ZERO: Self = 0.0;
    const ONE: Self = 1.0;
    
    fn add(self, other: Self) -> Self {
        self + other
    }
    
    fn multiply(self, other: Self) -> Self {
        self * other
    }
}

// Generic function with multiple trait bounds
fn sum_of_squares<T>(values: &[T]) -> T
where
    T: Numeric + Copy + std::fmt::Display,
{
    let mut sum = T::ZERO;
    for &value in values {
        let squared = value.square();
        println!("{}² = {}", value, squared);
        sum = sum.add(squared);
    }
    sum
}

// Trait with Self type
trait Cloneable {
    fn clone_self(&self) -> Self;
}

#[derive(Debug)]
struct Point {
    x: i32,
    y: i32,
}

impl Cloneable for Point {
    fn clone_self(&self) -> Self {
        Point { x: self.x, y: self.y }
    }
}

fn advanced_traits_demo() {
    println!("\n=== Advanced Traits Demo ===");
    
    // Custom iterator
    let mut counter = Counter::new(5);
    println!("Counter iterator:");
    while let Some(value) = counter.next() {
        println!("  {}", value);
    }
    
    // Associated constants
    println!("\nNumeric constants:");
    println!("i32 ZERO: {}, ONE: {}", i32::ZERO, i32::ONE);
    println!("f64 ZERO: {}, ONE: {}", f64::ZERO, f64::ONE);
    
    // Generic function with multiple bounds
    let integers = [1, 2, 3, 4];
    let int_sum = sum_of_squares(&integers);
    println!("\nSum of squares (integers): {}", int_sum);
    
    let floats = [1.5, 2.5, 3.5];
    let float_sum = sum_of_squares(&floats);
    println!("\nSum of squares (floats): {:.2}", float_sum);
    
    // Self type in traits
    let point = Point { x: 10, y: 20 };
    let cloned_point = point.clone_self();
    println!("\nOriginal point: {:?}", point);
    println!("Cloned point: {:?}", cloned_point);
}

advanced_traits_demo();

In [ ]:
// Trait objects and dynamic dispatch

trait Animal {
    fn name(&self) -> &str;
    fn make_sound(&self);
    
    fn introduce(&self) {
        println!("Hi, I'm {} and I go:", self.name());
        self.make_sound();
    }
}

struct Dog {
    name: String,
    breed: String,
}

impl Dog {
    fn new(name: String, breed: String) -> Self {
        Dog { name, breed }
    }
    
    fn breed(&self) -> &str {
        &self.breed
    }
}

impl Animal for Dog {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        println!("Woof! Woof!");
    }
    
    fn introduce(&self) {
        println!("Hi, I'm {}, a {} dog, and I go:", self.name, self.breed);
        self.make_sound();
    }
}

struct Cat {
    name: String,
    indoor: bool,
}

impl Cat {
    fn new(name: String, indoor: bool) -> Self {
        Cat { name, indoor }
    }
}

impl Animal for Cat {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        if self.indoor {
            println!("Meow meow (softly)");
        } else {
            println!("MEOW! HISS!");
        }
    }
}

struct Bird {
    name: String,
    species: String,
}

impl Bird {
    fn new(name: String, species: String) -> Self {
        Bird { name, species }
    }
}

impl Animal for Bird {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        match self.species.as_str() {
            "parrot" => println!("Squawk! Hello! Squawk!"),
            "crow" => println!("Caw! Caw!"),
            "canary" => println!("Tweet tweet!"),
            _ => println!("Chirp chirp!"),
        }
    }
}

// Function that works with any Animal trait object
fn animal_chorus(animals: &[Box<dyn Animal>]) {
    println!("🎵 Animal Chorus Time! 🎵");
    for (i, animal) in animals.iter().enumerate() {
        println!("\n{}. ", i + 1);
        animal.introduce();
    }
}

// Generic function (static dispatch)
fn introduce_animal<T: Animal>(animal: &T) {
    animal.introduce();
}

// Function returning trait object
fn create_random_animal(seed: u32) -> Box<dyn Animal> {
    match seed % 3 {
        0 => Box::new(Dog::new("Rex".to_string(), "Golden Retriever".to_string())),
        1 => Box::new(Cat::new("Whiskers".to_string(), true)),
        _ => Box::new(Bird::new("Polly".to_string(), "parrot".to_string())),
    }
}

fn trait_objects_demo() {
    println!("\n=== Trait Objects and Dynamic Dispatch ===");
    
    // Create different animals
    let dog = Dog::new("Buddy".to_string(), "Labrador".to_string());
    let cat = Cat::new("Mittens".to_string(), false);
    let bird = Bird::new("Tweety".to_string(), "canary".to_string());
    
    // Static dispatch (compile-time)
    println!("Static dispatch:");
    introduce_animal(&dog);
    introduce_animal(&cat);
    introduce_animal(&bird);
    
    // Dynamic dispatch (runtime)
    let animals: Vec<Box<dyn Animal>> = vec![
        Box::new(dog),
        Box::new(cat),
        Box::new(bird),
    ];
    
    println!("\nDynamic dispatch:");
    animal_chorus(&animals);
    
    // Creating animals at runtime
    println!("\n\nRandom animals:");
    for i in 0..5 {
        let animal = create_random_animal(i * 7 + 3);
        print!("Random animal {}: ", i + 1);
        animal.introduce();
        println!();
    }
}

trait_objects_demo();

In [ ]:
// TODO: Complete the media player system using traits

trait Playable {
    fn play(&self);
    fn pause(&self);
    fn stop(&self);
    fn duration(&self) -> u32; // in seconds
    fn title(&self) -> &str;
    
    // Default implementation
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("'{}' - Duration: {}:{:02}", self.title(), minutes, seconds);
    }
}

trait Seekable {
    fn seek_to(&self, position: u32);
    fn fast_forward(&self, seconds: u32) {
        println!("Fast forwarding {} seconds", seconds);
    }
    fn rewind(&self, seconds: u32) {
        println!("Rewinding {} seconds", seconds);
    }
}

#[derive(Debug)]
struct Song {
    title: String,
    artist: String,
    duration: u32,
    genre: String,
}

impl Song {
    fn new(title: String, artist: String, duration: u32, genre: String) -> Self {
        Song { title, artist, duration, genre }
    }
    
    fn artist(&self) -> &str {
        &self.artist
    }
    
    fn genre(&self) -> &str {
        &self.genre
    }
}

impl Playable for Song {
    fn play(&self) {
        println!("🎵 Playing song: '{}' by {}", self.title, self.artist);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused: '{}'", self.title);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped: '{}'", self.title);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
    
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("🎵 '{}' by {} [{}] - Duration: {}:{:02}", 
                self.title(), self.artist(), self.genre(), minutes, seconds);
    }
}

impl Seekable for Song {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("🎵 Seeking '{}' to {}:{:02}", self.title, minutes, seconds);
    }
}

#[derive(Debug)]
struct Podcast {
    title: String,
    host: String,
    episode: u32,
    duration: u32,
}

impl Podcast {
    fn new(title: String, host: String, episode: u32, duration: u32) -> Self {
        Podcast { title, host, episode, duration }
    }
    
    fn host(&self) -> &str {
        &self.host
    }
    
    fn episode(&self) -> u32 {
        self.episode
    }
}

impl Playable for Podcast {
    fn play(&self) {
        println!("🎙️  Playing podcast: '{}' Episode {} with {}", 
                self.title, self.episode, self.host);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused podcast: '{}' Episode {}", self.title, self.episode);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped podcast: '{}' Episode {}", self.title, self.episode);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
    
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("🎙️  '{}' Episode {} with {} - Duration: {}:{:02}", 
                self.title(), self.episode(), self.host(), minutes, seconds);
    }
}

impl Seekable for Podcast {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("🎙️  Seeking '{}' Episode {} to {}:{:02}", 
                self.title, self.episode, minutes, seconds);
    }
}

#[derive(Debug)]
struct AudioBook {
    title: String,
    author: String,
    narrator: String,
    chapter: u32,
    total_chapters: u32,
    duration: u32,
}

impl AudioBook {
    fn new(title: String, author: String, narrator: String, 
           chapter: u32, total_chapters: u32, duration: u32) -> Self {
        AudioBook { title, author, narrator, chapter, total_chapters, duration }
    }
}

impl Playable for AudioBook {
    fn play(&self) {
        println!("📚 Playing audiobook: '{}' by {} (Chapter {}/{})", 
                self.title, self.author, self.chapter, self.total_chapters);
        println!("    Narrated by {}", self.narrator);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused audiobook: '{}' Chapter {}", self.title, self.chapter);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped audiobook: '{}'", self.title);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
}

impl Seekable for AudioBook {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("📚 Seeking '{}' Chapter {} to {}:{:02}", 
                self.title, self.chapter, minutes, seconds);
    }
}

// Media player that works with any Playable + Seekable item
struct MediaPlayer {
    playlist: Vec<Box<dyn Playable>>,
    current_index: Option<usize>,
}

impl MediaPlayer {
    fn new() -> Self {
        MediaPlayer {
            playlist: Vec::new(),
            current_index: None,
        }
    }
    
    fn add_media(&mut self, media: Box<dyn Playable>) {
        self.playlist.push(media);
        if self.current_index.is_none() {
            self.current_index = Some(0);
        }
    }
    
    fn play_current(&self) {
        if let Some(index) = self.current_index {
            if let Some(media) = self.playlist.get(index) {
                media.play();
            }
        } else {
            println!("No media in playlist");
        }
    }
    
    fn next(&mut self) {
        if let Some(index) = self.current_index {
            if index + 1 < self.playlist.len() {
                self.current_index = Some(index + 1);
                println!("⏭️  Next track");
                self.play_current();
            } else {
                println!("End of playlist");
            }
        }
    }
    
    fn previous(&mut self) {
        if let Some(index) = self.current_index {
            if index > 0 {
                self.current_index = Some(index - 1);
                println!("⏮️  Previous track");
                self.play_current();
            } else {
                println!("Beginning of playlist");
            }
        }
    }
    
    fn show_playlist(&self) {
        println!("\n📋 Playlist:");
        for (i, media) in self.playlist.iter().enumerate() {
            let marker = if Some(i) == self.current_index { "▶️ " } else { "   " };
            print!("{}{}: ", marker, i + 1);
            media.info();
        }
        println!();
    }
}

fn media_player_demo() {
    println!("\n=== Media Player System Demo ===");
    
    let mut player = MediaPlayer::new();
    
    // Add different types of media
    player.add_media(Box::new(Song::new(
        "Bohemian Rhapsody".to_string(),
        "Queen".to_string(),
        355, // 5:55
        "Rock".to_string()
    )));
    
    player.add_media(Box::new(Podcast::new(
        "Rust Programming Podcast".to_string(),
        "Rustacean".to_string(),
        42,
        2700 // 45 minutes
    )));
    
    player.add_media(Box::new(AudioBook::new(
        "The Rust Programming Language".to_string(),
        "Steve Klabnik".to_string(),
        "AI Narrator".to_string(),
        1,
        20,
        1800 // 30 minutes
    )));
    
    player.add_media(Box::new(Song::new(
        "Thunderstruck".to_string(),
        "AC/DC".to_string(),
        292, // 4:52
        "Hard Rock".to_string()
    )));
    
    // Show playlist
    player.show_playlist();
    
    // Play through playlist
    println!("🎵 Starting playback:");
    player.play_current();
    
    println!();
    player.next();
    
    println!();
    player.next();
    
    println!();
    player.next();
    
    println!();
    player.previous();
    
    player.show_playlist();
}

media_player_demo();